# VQ VAE on CelebA
Training model and PixelCNN Sampler

Phase 1: Train VQVAE

Phase 2: Train PixelCNN

Phase 3: Sample images


Note: After some attemps I see that the model produce images a little bit blurry to i chose to make it deeper

## Phase 1: Train The variational Autoencoder

In [1]:
import itertools
import os
import sys

from classes.VQVAE import VQVAE
from classes.PixelCNN import PixelCNN,TfDistPixelCNN
from utils.callbacks import WandbImagesVQVAE, Save_VQVAE_Weights, Save_PixelCNN_Weights
from utils.functions import  map_vqvae_weights,pixelcnn_sample_vqvae
from imutils import build_montages


import tensorflow as tf
from tensorflow import keras
import numpy as np
import wandb
from wandb.keras import WandbCallback
from tensorflow.data import AUTOTUNE

import argparse
from os.path import join as opj
from imutils import paths
import tqdm
import glob
import json

In [2]:
#set the first GPU
os.environ["CUDA_VISIBLE_DEVICES"]="1"
print(os.environ.get("CUDA_VISIBLE_DEVICES"))

1


In [3]:
wandb.login()
#phase="VQ_VAE_Training"          #change
phase="PixelCNN_Training"

config={"dataset":"celebA", "type":"VQ-VAE","phase":phase}

images_dir=r"C:\Users\matte\Dataset\tor_vergata\Dataset\Img\img_align_celeba" #local
images_dir=r"/home/matteo/NeuroGEN/Dataset/Img/img_align_celeba"



Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: You can find your API key in your browser here: https://wandb.ai/authorize


wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [4]:
BS = 256
EPOCHS=240
INIT_LR=5e-5

config["BS"]=BS
config["EPOCHS"]=EPOCHS
config["INIT_LR"]=INIT_LR
config["nota"]="DGX"

## Dataloaders

In [5]:
def load_images(imagePath):
    # read the image from disk, decode it, resize it, and scale the
    # pixels intensities to the range [0, 1]
    image = tf.io.read_file(imagePath)
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, (128, 128)) / 255.0

    #eventually load other information like attributes here
    
    # return the image and the extra info
    
    
    return image

In [6]:
print("[INFO] loading image paths...")
imagePaths = list(paths.list_images(images_dir))


train_len=int(0.8*len(imagePaths))
val_len=int(0.1*len(imagePaths))
test_len=int(0.1*len(imagePaths))

train_imgs=imagePaths[:train_len]                                #      80% for training
val_imgs=imagePaths[train_len:train_len+val_len]                 #      10% for validation
test_imgs=imagePaths[train_len+val_len:]                         #      10% for testing

print(f"[TRAINING]\t {len(train_imgs)}\n[VALIDATION]\t {len(val_imgs)}\n[TEST]\t\t {len(test_imgs)}")

[INFO] loading image paths...
[TRAINING]	 138545
[VALIDATION]	 17318
[TEST]		 17319


In [7]:
#TRAINING 

train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .repeat()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

vs=len(val_imgs)//BS

## TEST

test_dataset = tf.data.Dataset.from_tensor_slices(test_imgs)
test_dataset = (test_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

2021-12-14 14:36:53.239416: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-12-14 14:36:53.722178: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79131 MB memory:  -> device: 0, name: NVIDIA A100-SXM-80GB, pci bus id: 0000:0f:00.0, compute capability: 8.0


## Model Definition

In [8]:
print(f"[INFO] Training VQ_VAE Model")

encoder_architecture=[(1,64),(1,128),(1,256),(3,256)]
decoder_architecture=[(3,256),(1,256),(1,128),(1,64)]
input_shape=(128,128,3)
latent_dim=512
num_embeddings=256


config["encoder_architecture"]=encoder_architecture
config["decoder_architecture"]=decoder_architecture
config["latent_dim"]=latent_dim
config["num_embeddings"]=num_embeddings
config["input_shape"]=input_shape

g=VQVAE(input_shape,latent_dim=latent_dim,num_embeddings=num_embeddings,train_variance=4,encoder_architecture=encoder_architecture,decoder_architecture=decoder_architecture)

print(g.encoder.summary())

print(g.decoder.summary())


[INFO] Training VQ_VAE Model
Model: "encoder"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 image_input (InputLayer)    [(None, 128, 128, 3)]     0         
                                                                 
 conv_res_block (ConvResBloc  (None, 64, 64, 64)       80320     
 k)                                                              
                                                                 
 conv_res_block_1 (ConvResBl  (None, 32, 32, 128)      386560    
 ock)                                                            
                                                                 
 conv_res_block_2 (ConvResBl  (None, 16, 16, 256)      1543168   
 ock)                                                            
                                                                 
 conv_res_block_3 (ConvResBl  (None, 8, 8, 256)        4334080   
 ock)                         

# Phase 1: Train the VQ-VAE model

## Callbacks

In [9]:
wandb.init(project="TorVergataExperiment-Generative",config=config)

wandb: Currently logged in as: matteoferrante (use `wandb login --relogin` to force relogin)


In [10]:
os.makedirs("models/vqvae_celeba",exist_ok=True)
model_check= Save_VQVAE_Weights("models/vqvae_celeba")



es=tf.keras.callbacks.EarlyStopping(
    monitor="loss",
    min_delta=0,
    patience=8,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)


callbacks=[
    WandbImagesVQVAE(test_dataset,sample=False),
    WandbCallback(),
    model_check,
    es,
]


## COMPILE AND TRAIN

In [15]:
g.compile(keras.optimizers.Adam(INIT_LR))
g.build(input_shape=(None,*input_shape))

In [ ]:
g.fit(train_dataset,validation_data=val_dataset,steps_per_epoch=ts,validation_steps=vs,epochs=EPOCHS,callbacks=callbacks)


('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
('You must install pydot (`pip install pydot`) and install graphviz (see instructions at https://graphviz.gitlab.io/download/) ', 'for plot_model/model_to_dot to work.')
Epoch 1/240


2021-12-14 14:37:58.440055: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8300
2021-12-14 14:38:00.186729: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


541/541 [==============================] - ETA: 0s - loss: 0.0689 - reconstruction_loss: 0.0105 - vqvae_loss: 0.0535

2021-12-14 14:41:00.760040: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


541/541 [==============================] - 187s 327ms/step - loss: 0.0689 - reconstruction_loss: 0.0105 - vqvae_loss: 0.0536 - val_val_reconstruction_loss: 0.0059
Epoch 2/240
541/541 [==============================] - ETA: 0s - loss: 0.0395 - reconstruction_loss: 0.0051 - vqvae_loss: 0.0343

...

541/541 [==============================] - 172s 318ms/step - loss: 0.0069 - reconstruction_loss: 0.0022 - vqvae_loss: 0.0046 - val_val_reconstruction_loss: 0.0023
Epoch 27/240
541/541 [==============================] - ETA: 0s - loss: 0.0068 - reconstruction_loss: 0.0022 - vqvae_loss: 0.0046

2021-12-14 15:55:34.482417: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


541/541 [==============================] - 172s 318ms/step - loss: 0.0068 - reconstruction_loss: 0.0022 - vqvae_loss: 0.0046 - val_val_reconstruction_loss: 0.0022
Epoch 28/240
541/541 [==============================] - ETA: 0s - loss: 0.0068 - reconstruction_loss: 0.0022 - vqvae_loss: 0.0046

2021-12-14 15:58:26.256615: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being cached. In order to avoid unexpected truncation of the dataset, the partially cached contents of the dataset  will be discarded. This can happen if you have an input pipeline similar to `dataset.cache().take(k).repeat()`. You should use `dataset.take(k).cache().repeat()` instead.


541/541 [==============================] - 172s 318ms/step - loss: 0.0068 - reconstruction_loss: 0.0022 - vqvae_loss: 0.0046 - val_val_reconstruction_loss: 0.0022
Epoch 29/240
 46/541 [=>............................] - ETA: 2:30 - loss: 0.0069 - reconstruction_loss: 0.0022 - vqvae_loss: 0.0047

# Phase 2: Train the PixelCNN sampler



In [ ]:
def map_models_weights(model_dir):
    files=os.listdir(model_dir)
    d={}
    for f in files:
        if "encoder" in f:
            d["encoder"]=opj(model_dir,f)
        elif ("generator" in f) or ("decoder" in f):
            d["decoder"]=opj(model_dir,f)
        elif "embeddings" in f:
            d["embeddings"]=opj(model_dir,f)
    return d


In [ ]:

#model=VQVAE((28,28,1),latent_dim=16,num_embeddings=128,train_variance=4)

weights=map_models_weights("models/vqvae_celeba")

print(weights)
g.load_weights(weights["encoder"],weights["embeddings"],weights["decoder"])


BS=64

## Using smaller batch size while predict

A memory bug?

In [ ]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_imgs)
train_dataset = (train_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

ts=len(train_imgs)//BS

##VALIDATION

val_dataset = tf.data.Dataset.from_tensor_slices(val_imgs)
val_dataset = (val_dataset
    .shuffle(1024)
    .map(load_images, num_parallel_calls=AUTOTUNE)
    .cache()
    .batch(BS)
    .prefetch(AUTOTUNE)
)

In [ ]:
encoded_outputs=[]

for batch in tqdm.tqdm(train_dataset):
    encoded_outputs.append(g.encoder.predict_on_batch(batch))

#encoded_outputs = g.encoder.predict(train_dataset)


In [ ]:
encoded_outputs[0][0].shape

In [ ]:
codebook_indices=[]
for b in tqdm.tqdm(range(len(train_dataset))):

    #flat the outputs
    batch_encoded=np.array(encoded_outputs[b])
    flat_enc_outputs=batch_encoded.reshape(-1,batch_encoded.shape[-1])
    code_index=g.vq_layer.get_code_indices(flat_enc_outputs).numpy()
    codebook_indices.append(list(code_index))




In [ ]:
codebook_indices = list(itertools.chain.from_iterable(codebook_indices))
codebook_indices = np.array(codebook_indices)
print(codebook_indices.shape)

square_dim=batch_encoded.shape[1]

codebook_indices = codebook_indices.reshape(-1,square_dim,square_dim)
print(f"Shape of the training data for PixelCNN: {codebook_indices.shape}")

input_shape=(codebook_indices.shape[1],codebook_indices.shape[2])

In [ ]:
pixel_cnn = PixelCNN(input_dim=input_shape, n_embeddings=256, n_residual=8, n_convlayer=4)
opt = keras.optimizers.Adam(3e-4)
pixel_cnn.compile(loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True), optimizer=opt,metrics="accuracy")

wandb.init(project="TorVergataExperiment-Generative",config=config)

In [ ]:
es = tf.keras.callbacks.EarlyStopping(
    monitor="val_loss",
    min_delta=0,
    patience=15,
    verbose=0,
    mode="auto",
    baseline=None,
    restore_best_weights=True,
)
callbacks = [WandbCallback(), es]


In [ ]:
pixel_cnn_dir="models/vqvae_celeba/pixelcnn"
os.makedirs(pixel_cnn_dir,exist_ok=True)

pixel_cnn.save_dict(opj(pixel_cnn_dir,"dict.json"))

pixel_cnn.fit(codebook_indices, codebook_indices, batch_size=BS, validation_split=0.1, epochs=30,
              callbacks=callbacks)
pixel_cnn.model.save_weights(opj(pixel_cnn_dir,"pixelcnn_vqvae_celeba_weights.h5"))



# Phase 3: Sample the VQ VAE

In [ ]:
weights=map_models_weights("models/vqvae_celeba")

print(weights)
g.load_weights(weights["encoder"],weights["embeddings"],weights["decoder"])

BS=64



In [ ]:
pixel_cnn_dir="models/vqvae_celeba/pixelcnn"


config=glob.glob(opj(pixel_cnn_dir,"*.json"))[0]
weights=glob.glob(opj(pixel_cnn_dir,"*.h5"))[0]

print(f"[CONFIG] PixelCNN {config}")
f=open(config,"r")

config=json.load(f)

input_dim=config["input_dim"]
n_emb=config["num_embeddings"]
n_res=config["n_residual"]
n_conv=config["n_convlayer"]
ksize=config["ksize"]
p=PixelCNN(input_dim=input_dim,n_embeddings=n_emb,n_residual=n_res,n_convlayer=n_conv,ksize=ksize)

p.model.load_weights(weights)

In [ ]:
generated_samples=pixelcnn_sample_vqvae(100,p.model,g)

images = generated_samples * 255.
#images = np.repeat(images, 3, axis=-1)
vis = build_montages(images, (128, 128), (10, 10))[0]

log = {f"image_sampled": wandb.Image(vis)}
wandb.log(log)


In [ ]:
import matplotlib.pyplot as plt
plt.imshow(vis)